In [1]:
#A Colab pro environment should have >20Gb of total memory.
from psutil import virtual_memory
colab_pro = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(colab_pro))

if colab_pro < 20:
  print('Not using a high-RAM runtime')
  # train model with lower settings
else:
  print('You are using a high-RAM runtime!')
  # train model with higher settings

Your runtime has 89.6 gigabytes of available RAM

You are using a high-RAM runtime!


In [4]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Fri Jan  6 01:05:19 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  A100-SXM4-40GB      Off  | 00000000:00:04.0 Off |                    0 |
| N/A   30C    P0    51W / 400W |      0MiB / 40536MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
!pip install adapter-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 44.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 81.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 21.9 MB/s eta 0:00:00


In [6]:
from transformers import  AutoTokenizer, AutoModelForCausalLM


tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-j-6B")


In [7]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained("EleutherAI/gpt-j-6B")


In [8]:
!unzip bbc.zip

Archive:  bbc.zip
   creating: bbc/
   creating: bbc/business/
  inflating: bbc/business/001.txt    
  inflating: bbc/business/002.txt    
  inflating: bbc/business/003.txt    
  inflating: bbc/business/004.txt    
  inflating: bbc/business/005.txt    
  inflating: bbc/business/006.txt    
  inflating: bbc/business/007.txt    
  inflating: bbc/business/008.txt    
  inflating: bbc/business/009.txt    
  inflating: bbc/business/010.txt    
  inflating: bbc/business/011.txt    
  inflating: bbc/business/012.txt    
  inflating: bbc/business/013.txt    
  inflating: bbc/business/014.txt    
  inflating: bbc/business/015.txt    
  inflating: bbc/business/016.txt    
  inflating: bbc/business/017.txt    
  inflating: bbc/business/018.txt    
  inflating: bbc/business/019.txt    
  inflating: bbc/business/020.txt    
  inflating: bbc/business/021.txt    
  inflating: bbc/business/022.txt    
  inflating: bbc/business/023.txt    
  inflating: bbc/business/024.txt    
  inflating: bbc/business

In [10]:
import pandas as pd
from pathlib import Path



In [11]:
DATA_DIR = Path('data')
files = (DATA_DIR / 'bbc').glob('**/*.txt')
bbc_articles = []
for i, file in enumerate(sorted(list(files))):
    with file.open(encoding='latin1') as f:
        lines = f.readlines()
        body = ' '.join([l.strip() for l in lines[1:]]).strip()
        bbc_articles.append(body)

In [17]:
#bbc_articles[:5]

In [18]:
news = pd.DataFrame(bbc_articles, columns=['headline_text'])

In [19]:
news

,headline_text
0,Quarterly profits at US media giant TimeWarner...
1,The dollar has hit its highest level against t...
2,The owners of embattled Russian oil giant Yuko...
3,British Airways has blamed high fuel prices fo...
4,Shares in UK drinks and food firm Allied Domec...
...,...
2220,BT is introducing two initiatives to help beat...
2221,Computer users across the world continue to ig...
2222,A new European directive could put software wr...
2223,The man making sure US computer networks are s...


In [21]:
from sklearn.model_selection import train_test_split
X_train, X_test= train_test_split(news.headline_text.tolist(),test_size=0.20, random_state=42)
len(X_train), len(X_test)

(1780, 445)

In [22]:
with open('train_dataset.txt','w') as f:
  for line in X_train:
    f.write(line)
    f.write("\n")


In [23]:
with open('test_dataset.txt','w') as f:
  for line in X_test:
    f.write(line)
    f.write("\n")

In [23]:
from transformers import AutoModelForCausalLM, AutoTokenizer, GPTJ
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-j-6B")

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("gpt2",pad_token='<pad>')

train_path = 'train_dataset.txt'
test_path = 'test_dataset.txt'

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embedding are fine-tuned or trained.


In [ ]:
from transformers import TextDataset,DataCollatorForLanguageModeling

def load_dataset(train_path,test_path,tokenizer):
    train_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=train_path,
          block_size=4)
     
    test_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=test_path,
          block_size=4)   
    
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    )
    return train_dataset,test_dataset,data_collator

train_dataset,test_dataset,data_collator = load_dataset(train_path,test_path,tokenizer)

/usr/local/lib/python3.8/dist-packages/transformers/modeling_auto.py:781: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


OSError: ignored

In [ ]:
from transformers import Trainer, TrainingArguments,AutoModelWithLMHead

model = AutoModelWithLMHead.from_pretrained("gpt2")


training_args = TrainingArguments(
    output_dir="./headliner", #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=1, # number of training epochs
    per_device_train_batch_size=16, # batch size for training
    per_device_eval_batch_size=4,  # batch size for evaluation
    eval_steps = 40, # Number of update steps between two evaluations.
    save_steps=80, # after # steps model is saved 
    warmup_steps=50,# number of warmup steps for learning rate scheduler
    )


trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    prediction_loss_only=True,
)

/usr/local/lib/python3.8/dist-packages/transformers/modeling_auto.py:781: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


Downloading:   0%|          | 0.00/548M [00:00<?, ?B/s]

/usr/local/lib/python3.8/dist-packages/transformers/trainer.py:265: FutureWarning: Passing `prediction_loss_only` as a keyword argument is deprecated and won't be possible in a future version. Use `args.prediction_loss_only` instead.
  warnings.warn(


In [ ]:
trainer.train()

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/13149 [00:00<?, ?it/s]

{'loss': 5.5363857421875, 'learning_rate': 4.8282311626841746e-05, 'epoch': 0.03802570537683474, 'total_flos': 23892443136000, 'step': 500}
{'loss': 5.39728515625, 'learning_rate': 4.637376898999924e-05, 'epoch': 0.07605141075366947, 'total_flos': 47784886272000, 'step': 1000}
{'loss': 5.3970634765625, 'learning_rate': 4.4465226353156733e-05, 'epoch': 0.11407711613050422, 'total_flos': 71677329408000, 'step': 1500}
{'loss': 5.336720703125, 'learning_rate': 4.255668371631422e-05, 'epoch': 0.15210282150733895, 'total_flos': 95569772544000, 'step': 2000}


KeyboardInterrupt: ignored

In [ ]:
trainer.save_model()

In [ ]:
from transformers import pipeline

headliner = pipeline('text-generation',
                model='./headliner', 
                tokenizer='gpt2',
                config={'max_length':8})

In [ ]:
def get_headline(headliner_pipeline, seed_text="News"):
  return headliner_pipeline(seed_text)[0]['generated_text'].split('\n')[0]

In [ ]:
get_headline(headliner, seed_text="News")

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


'News is all about the same.  My friends, relatives and friends from all walks of life will have a chance to play through a whole album with each new story when they do have one. "For our audience, it\'s a personal experience -'

In [ ]:
get_headline(headliner, seed_text="China decides")

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


'China decides he wants to do it, or not to put in the cart before the door "to meet all of the demands of the people who are trying to change it." But there have been concerns among many in the party about what is actually going'

In [ ]:
get_headline(headliner, seed_text="Wildfire")

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


'Wildfire from the front line.  "We will be looking at a few changes of the way the club are tackling this and this could be considered from the bench and it has been a little bit overawed for us last season," he said.'

In [ ]:
get_headline(headliner, seed_text="City Council")

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


'City Council on Sunday, in the Commons and in the Commons, after a "rattled" decision of its own and council chancellor John Prescott to cut back on the chancellor having made the case that Britain wanted more money to be spent on spending. This'

In [ ]:
get_headline(headliner, seed_text="Amazon's leadership principles are")

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


'Amazon\'s leadership principles are set out by the government," said a senior adviser at the bank to the IMF on global banking. However, Iwanis argues that the IMF should also take responsibility to monitor economic growth and the future direction it is considering and'

In [ ]:
# Tokenizer